In [ ]:
import torch
import numpy as np
import random
import time
from tqdm import tqdm  # 
from pathlib import Path


seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
# torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = True

if torch.cuda.is_available():
    print(f"cuda:True")
    for i in range(torch.cuda.device_count()):
        device = torch.device(f"cuda:{i}")
        properties = torch.cuda.get_device_properties(device)
        print(f"GPU {i}:{properties.name}:{properties.total_memory/1024/1024/1024:.2f}GB")
else:
    print(f"cuda:False")
    device = torch.device("cpu")

    
import sys
import os
from models_LN import FNN_LN, dataset, one_hot, cross_entropy_loss, linear_d, tanh, tanh_d, sign, sign_d, sigmoid, sigmoid_d, relu, relu_d, relu6, relu6_d, softmax, softmax_d

torch.set_grad_enabled(False)


cuda:True
GPU 0:NVIDIA RTX A5000:23.99GB


In [2]:

class Config:
    def __init__(self):
        self.n_input = 28 * 28  # 
        self.n_hidd = 16     # 
        self.n_out = 10      # 

        self.nL_hidd = 2

        self.sc_forward = 1
        self.sc_bias = 0
        self.sc_back = 1

        self.f = tanh #  relu tanh
        self.fd = tanh_d # relu_d tanh_d

        self.train_eta_global = 1e-3  #
        self.train_n_epochs = 100     # 
        self.train_batch_size = 500     # 
        self.train_task = "MNIST" #  "MNIST" "FMNIST" "CIFAR10"
        self.train_tmethod = ''     # 'FA','PC','LRA','BP'

        self.biasLearning = False
        
        self.EP_It2sta = 8
        self.EP_nudge = 1


config = Config()

train_loader, valid_loader, test_loader, config = dataset(config)
train_loader_gpu = [(data.to(device), target.to(device)) for data, target in train_loader]
valid_loader_gpu = [(data.to(device), target.to(device)) for data, target in valid_loader]
test_loader_gpu = [(data.to(device), target.to(device)) for data, target in test_loader]


In [3]:
ParaName = f'First_com_BP_FA_EP_512_t52'
Einfo = ''

res_path = Path(f"./Res_com/{ParaName}-{config.train_task}-0")
M_range = ['EP-s', 'BP'] # 'EP-s', 'BP'
ParaRange = [3,2] # 2, 3
multitest = 5 
config.save = True
config.f = tanh
config.fd = tanh_d
config.ff = softmax
config.ffd = softmax_d
# -=--------------------------
config.train_tmethod = 'EP'
config.nL_hidd = 2

config.n_hidd = 512     # 
config.train_n_epochs = 50  # 
config.EP_symm_W = True
config.adam = True
config.EP_b_learn = False
config.EP_f_sc = 1
config.EP_b_sc = 0.1
config.EP_nudge = 0.1
# -=--------------------------
res_path.mkdir(parents=True, exist_ok=True)
res = np.zeros((len(M_range),len(ParaRange),(multitest),5,config.train_n_epochs))

In [4]:
current_time = time.strftime("%Y%m%d-%H%M%S", time.localtime())
print(f'Current Time: {current_time}')
with open(res_path/"res.txt", "a") as f: 
    f.write(f"===================================================================")       
    f.write(f"Start Time: {current_time}\r\n")

for iM in range( len(M_range) ):
    for iPara in range( len(ParaRange) ):
        # if M_range[iM] == True and iPara>4: continue 
        # if M_range[iM] == False and iPara<2: continue
        for  imultitest in range((multitest)):
            if M_range[iM]=='EP-s':
                config.EP_symm_W = True
                config.train_tmethod = 'EP'
            elif M_range[iM]=='EP-a':
                config.EP_symm_W = False
                config.train_tmethod = 'EP'
            else:
                config.train_tmethod = M_range[iM]
            config.nL_hidd = ParaRange[iPara]
            config.EP_b_sc = 0.1 if config.nL_hidd>4 else 0.01

            #-----------------
            config.EP_It2sta = 10 if config.nL_hidd==2 else 18
            config.EP_It2sta2 = 5 if config.nL_hidd==2 else 10
            # ---------------------------------
            model = FNN_LN(config).to(device)

            taskinfo = f'{config.train_tmethod}_L{config.nL_hidd}_{config.train_task}_{ParaName}-EP_b_sc{M_range[iM]}-EP_It2sta{config.EP_It2sta}-{Einfo}'
            print(taskinfo)

            ## Train

            for epoch in range(config.train_n_epochs):
                losssum = 0
                train_correct = 0
                train_total = 0
                random.shuffle(train_loader_gpu)
                with tqdm(total=len(train_loader_gpu), desc=f'Epoch {epoch + 1}/{config.train_n_epochs}: ', unit='batch', ncols=90, mininterval=1, colour='blue') as pbar:
                    for i, (images, labels) in enumerate(train_loader_gpu):
                        # 
                        images = images.view(-1, config.n_input)
                        labels_one_hot = one_hot(labels, config.n_out, device=device)
                        # 
                        outputs = model.forward(images)
                        _, predicted = torch.max(outputs, 1)
                        # 
                        loss = cross_entropy_loss(outputs, labels_one_hot)
                        # loss = loss.to('cpu')
                        losssum += loss.item()
                        #
                        outputs = model.backward(images, labels_one_hot, outputs)
                        # 
                        model.update_weights_adam() if config.adam else model.update_weights() #model.update_weights_adam() model.update_weights()
                        #  
                        if i % 10 == 0: 
                            pbar.set_postfix({'loss': f'{losssum/(i+1):.6f}'})  
                            pbar.update(10)

                        train_total += labels.size(0)
                        train_correct += (predicted == labels).sum().item()

                train_accuracy = train_correct / train_total

                # test set -------------------------------------------------------------
                test_correct = 0
                test_total = 0

                for images, labels in test_loader_gpu:
                    images = images.view(-1, config.n_input)
                    # 
                    outputs = model.forward(images)

                    _, predicted = torch.max(outputs, 1)
                    test_total += labels.size(0)
                    test_correct += (predicted == labels).sum().item()

                test_accuracy = test_correct / test_total

                res[iM][iPara][imultitest][0][epoch] = losssum
                res[iM][iPara][imultitest][1][epoch] = train_accuracy
                res[iM][iPara][imultitest][2][epoch] = test_accuracy

                current_time = time.strftime("%Y%m%d-%H%M%S", time.localtime())
                print(f'\t Loss: {res[iM][iPara][imultitest][0][epoch] :.3f},\t'
                    f'Train : {res[iM][iPara][imultitest][1][epoch] * 100:.2f}%({np.max(res[iM][iPara][imultitest][1]) * 100:.2f}%),\t'
                    f'Test : {res[iM][iPara][imultitest][2][epoch] * 100:.2f}%({np.max(res[iM][iPara][imultitest][2]) * 100:.2f}%),\t'
                    f'Current Time: {current_time}')
                
                # if 1-train_accuracy <1e-4:
                #     break
            
            # best result
            print(f"{taskinfo} Time: {current_time}, Epochs: {config.train_n_epochs}, Learning Rate: {config.train_eta_global}, "
                f"Best: {max(res[iM][iPara][imultitest][1]) * 100:.2f}%  {max(res[iM][iPara][imultitest][2]) * 100:.2f}%\n")
            
        # if config.save: 
        for attempt in range(5):
            try:
                with open(res_path/"res.txt", "a") as f:
                    trainacct = np.max(res[iM][iPara][:,1,:], axis=-1)
                    testacct = np.max(res[iM][iPara][:,2,:], axis=-1)
                    f.write(f"{taskinfo} Time: {current_time}: "
                            f"train: {np.mean(trainacct) * 100:.2f}%+-{np.std(trainacct, ddof=1)*100:.2f}% ({np.min(trainacct) * 100:.2f}% {np.max(trainacct) * 100:.2f}%)"
                            f"\t test: {np.mean(testacct) * 100:.2f}%+-{np.std(testacct, ddof=1)*100:.2f}% ({np.min(testacct) * 100:.2f}% {np.max(testacct) * 100:.2f}%)\n")

                    for attr_name in dir(config):
                        if not attr_name.startswith('__') and not callable(getattr(config, attr_name)):  
                            attr_value = getattr(config, attr_name)
                            f.write(f'{attr_name}: {attr_value}  ')  
                    f.write(f'\n\n')  

                break  # 
            except PermissionError:
                if attempt < 5 - 1:
                    print(f"Permission denied, retrying in {5} seconds...")
                    time.sleep(5)
                else:
                    raise  #

Current Time: 20250721-111348
EP_L3_MNIST_First_com_BP_FA_EP_512_t52-EP_b_scEP-s-EP_It2sta18-


Epoch 1/50:   0%|                                              | 0/120 [00:00<?, ?batch/s]d:\GPUS2share\aEP\models_LN.py:247: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.y = torch.tensor(self.z, device=self.device)
Epoch 1/50: 100%|█████████████████████| 120/120 [00:02<00:00, 41.01batch/s, loss=0.384623]


	 Loss: 44.933,	Train : 88.77%(88.77%),	Test : 93.35%(93.35%),	Current Time: 20250721-111352


Epoch 2/50: 100%|█████████████████████| 120/120 [00:02<00:00, 49.05batch/s, loss=0.185961]


	 Loss: 21.935,	Train : 94.60%(94.60%),	Test : 95.40%(95.40%),	Current Time: 20250721-111355


...

	 Loss: 0.014,	Train : 100.00%(100.00%),	Test : 98.14%(98.26%),	Current Time: 20250721-113326


Epoch 49/50: 100%|███████████████████| 120/120 [00:00<00:00, 373.99batch/s, loss=0.000105]


	 Loss: 0.013,	Train : 100.00%(100.00%),	Test : 98.15%(98.26%),	Current Time: 20250721-113326


Epoch 50/50: 100%|███████████████████| 120/120 [00:00<00:00, 328.57batch/s, loss=0.057122]

	 Loss: 6.860,	Train : 98.41%(100.00%),	Test : 96.94%(98.26%),	Current Time: 20250721-113326
BP_L2_MNIST_First_com_BP_FA_EP_512_t52-EP_b_scBP-EP_It2sta10- Time: 20250721-113326, Epochs: 50, Learning Rate: 0.001, Best: 100.00%  98.26%



In [5]:

# ---------------------------
current_time = time.strftime("%Y%m%d-%H%M%S", time.localtime())
if config.save: 
    save_dict = {
        "M_range": M_range,
        "ParaRange": ParaRange,
        "multitest": multitest,
        "train_task": config.train_task,
        "res": res,
    }
    np.savez_compressed(res_path / f"{taskinfo}_{current_time}_res",  **save_dict)
        



In [6]:
for iM in range( len(M_range) ):
    for iPara in range( len(ParaRange) ):        
        trainacct = np.max(res[iM][iPara][:,1,:], axis=-1)
        
        testacct = np.max(res[iM][iPara][:,2,:], axis=-1)
        print(f"{M_range[iM]}-{ParaRange[iPara]} Time: {current_time}: "
                f"train: {np.mean(trainacct) * 100:.2f}%+-{np.std(trainacct, ddof=1)*100:.2f}% ({np.min(trainacct) * 100:.2f}% {np.max(trainacct) * 100:.2f}%)"
                f"\t test: {np.mean(testacct) * 100:.2f}%+-{np.std(testacct, ddof=1)*100:.2f}% ({np.min(testacct) * 100:.2f}% {np.max(testacct) * 100:.2f}%)\n")
        


print('0')


EP-s-3 Time: 20250721-113326: train: 100.00%+-0.00% (100.00% 100.00%)	 test: 98.36%+-0.06% (98.30% 98.45%)

EP-s-2 Time: 20250721-113326: train: 100.00%+-0.00% (100.00% 100.00%)	 test: 98.30%+-0.07% (98.21% 98.41%)

BP-3 Time: 20250721-113326: train: 100.00%+-0.00% (100.00% 100.00%)	 test: 98.36%+-0.08% (98.28% 98.47%)

BP-2 Time: 20250721-113326: train: 100.00%+-0.00% (100.00% 100.00%)	 test: 98.29%+-0.05% (98.26% 98.37%)

0
